# `pyLDAvis.sklearn`

pyLDAvis now also supports LDA application from scikit-learn. Let's take a look into this in more detail. We will be using 20 newsgroups dataset as provided by scikit-learn.

In [3]:
from __future__ import print_function

In [5]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF


## Load 20 newsgroups dataset

First, the 20 newsgroups dataset available in sklearn are loaded. The headers, footers and quotes are removed, as always.

In [11]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def strip_links(x):
    stripped = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x)
    return stripped

def remove_stopwords(x):
    word_tokens = word_tokenize(x)
    filtered = [w for w in word_tokens if w.lower() not in stop_words if w.isalpha()]
    return " ".join(filtered)

df = pd.read_json('../data/all.json')
df["likes_normalized"] = df["likes"] / df["likes"].max()
df["replies_normalized"] = df["replies"] / df["replies"].max()
df["retweets_normalized"] = df["retweets"] / df["retweets"].max()
df = df[(df["likes_normalized"] > 0.05) | (df["replies_normalized"] > 0.05) | (df["retweets_normalized"] > 0.05)]
# df = df[(df["likes_normalized"] < 0.0001) | (df["replies_normalized"] < 0.0001) | (df["retweets_normalized"] < 0.0001)]
df['text'] = df['text'].apply(lambda x: x.replace('http', ' http'))
df['text'] = df['text'].apply(strip_links)
df['text'] = df['text'].apply(remove_stopwords)
print(len(df))

3898


In [12]:
docs_raw = df['text'].tolist()


## Convert to document-term matrix

Next, the raw documents are converted into document-term matrix, possibly as raw counts or TF-IDF form.

In [13]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
#                                 ngram_range=[1,3],
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(3898, 708)


In [14]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(3898, 708)


/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


## Fit Latent Dirichlet Allocation models

Finally, the LDA models are fitted.

In [15]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
# lda_tfidf = LatentDirichletAllocation(n_topics=10, random_state=0)
# lda_tfidf.fit(dtm_tfidf)
lda_tfidf = NMF(n_components=30, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(dtm_tfidf)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


## Visualzing the models with pyLDAvis

In [16]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/sklearn.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambd

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      17.961659        1       1 -0.027120  0.138802
25      5.995005        1       2 -0.251856 -0.021799
6       4.906196        1       3 -0.278756 -0.005299
4       4.348481        1       4 -0.062317  0.083867
8       4.040462        1       5 -0.015551  0.103006
12      3.791100        1       6  0.064387  0.055371
7       3.638062        1       7  0.036465  0.066537
5       3.610748        1       8  0.044106 -0.037486
18      3.339064        1       9 -0.225250 -0.027376
2       3.295082        1      10  0.042527  0.032625
24      3.035083        1      11  0.093347 -0.024710
17      2.995162        1      12 -0.219138  0.018541
23      2.944989        1      13  0.002435  0.134440
15      2.913381        1      14  0.081452 -0.026305
29      2.898293        1      15  0.111641  0.112459
9       2.747067        1      16  0.117117  0.083176
3       2.585162        1      17  0.087799  0.068627
13      2.534038        1      18  0.025747  0.063843
1       2.477247        1      19  0.093391  0.057990
16      2.474715        1      20  0.086726  0.024729
27      2.358789        1      21 -0.029342 -0.061436
14      2.283384        1      22  0.063173  0.054953
11      2.185911        1      23  0.057208 -0.226750
20      1.990066        1      24 -0.060336 -0.012234
10      1.866969        1      25  0.014193 -0.106443
21      1.682160        1      26  0.011315  0.034482
19      1.500566        1      27 -0.018325 -0.035755
22      1.493659        1      28  0.047209 -0.220720
28      1.358115        1      29  0.045718 -0.032532
26      0.749384        1      30  0.062035 -0.294601, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
636   Default  419.000000        trump  419.000000  30.0000  30.0000
219   Default  138.000000        going  138.000000  29.0000  29.0000
335   Default  125.000000         like  125.000000  28.0000  28.0000
465   Default  109.000000    president  109.000000  27.0000  27.0000
311   Default  103.000000         know  103.000000  26.0000  26.0000
394   Default  126.000000      mueller  126.000000  25.0000  25.0000
480   Default  117.000000        putin  117.000000  24.0000  24.0000
441   Default  101.000000       people  101.000000  23.0000  23.0000
222   Default  110.000000         good  110.000000  22.0000  22.0000
401   Default   94.000000         need   94.000000  21.0000  21.0000
328   Default   96.000000          let   96.000000  20.0000  20.0000
620   Default   90.000000         time   90.000000  19.0000  19.0000
524   Default  106.000000      russian  106.000000  18.0000  18.0000
512   Default   90.000000        right   90.000000  17.0000  17.0000
530   Default   80.000000          say   80.000000  16.0000  16.0000
357   Default   85.000000         make   85.000000  15.0000  15.0000
54    Default   76.000000    billboard   76.000000  14.0000  14.0000
631   Default   82.000000      treason   82.000000  13.0000  13.0000
567   Default  102.000000       source  102.000000  12.0000  12.0000
664   Default   70.000000         want   70.000000  11.0000  11.0000
99    Default   72.000000       coming   72.000000  10.0000  10.0000
613   Default   66.000000        think   66.000000   9.0000   9.0000
98    Default   82.000000        comey   82.000000   8.0000   8.0000
586   Default   74.000000        story   74.000000   7.0000   7.0000
506   Default   77.000000  republicans   77.000000   6.0000   6.0000
492   Default   63.000000       really   63.000000   5.0000   5.0000
681   Default   76.000000        white   76.000000   4.0000   4.0000
610   Default   60.000000       thanks   60.000000   3.0000   3.0000
269   Default   63.000000        house   63.000000   2.0000   2.0000
387   Default   61.000000        money   61.000000   1.0000   1.0000
...       ...        

### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [17]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      11.110392        1       1  0.047601  0.063875
9      11.002602        1       2  0.022207 -0.331416
0      10.719315        1       3 -0.265995 -0.091210
6      10.525247        1       4 -0.205064  0.334203
3      10.112650        1       5  0.271136 -0.287787
5       9.797891        1       6  0.381220 -0.042443
2       9.776977        1       7  0.048385  0.362579
7       9.442356        1       8 -0.276083 -0.379979
1       9.061331        1       9 -0.384467  0.108621
4       8.451239        1      10  0.361060  0.263556, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
480   Default   167.000000          putin   167.000000  30.0000  30.0000
311   Default   146.000000           know   146.000000  29.0000  29.0000
524   Default   190.000000        russian   190.000000  28.0000  28.0000
681   Default   120.000000          white   120.000000  27.0000  27.0000
269   Default   119.000000          house   119.000000  26.0000  26.0000
636   Default  1340.000000          trump  1340.000000  25.0000  25.0000
98    Default   123.000000          comey   123.000000  24.0000  24.0000
387   Default   107.000000          money   107.000000  23.0000  23.0000
465   Default   152.000000      president   152.000000  22.0000  22.0000
54    Default   102.000000      billboard   102.000000  21.0000  21.0000
335   Default   194.000000           like   194.000000  20.0000  20.0000
530   Default    95.000000            say    95.000000  19.0000  19.0000
222   Default   120.000000           good   120.000000  18.0000  18.0000
567   Default   121.000000         source   121.000000  17.0000  17.0000
512   Default   112.000000          right   112.000000  16.0000  16.0000
401   Default   132.000000           need   132.000000  15.0000  15.0000
328   Default   109.000000            let   109.000000  14.0000  14.0000
357   Default    82.000000           make    82.000000  13.0000  13.0000
223   Default    78.000000            gop    78.000000  12.0000  12.0000
291   Default    72.000000  investigation    72.000000  11.0000  11.0000
224   Default    71.000000            got    71.000000  10.0000  10.0000
613   Default    77.000000          think    77.000000   9.0000   9.0000
506   Default    76.000000    republicans    76.000000   8.0000   8.0000
394   Default   174.000000        mueller   174.000000   7.0000   7.0000
441   Default   122.000000         people   122.000000   6.0000   6.0000
579   Default    74.000000          state    74.000000   5.0000   5.0000
219   Default   203.000000          going   203.000000   4.0000   4.0000
586   Default    65.000000          story    65.000000   3.0000   3.0000
631   Default    66.000000        treason    66.000000   2.0000   2.0000
527   Default    66.000000           said    66.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
591   Topic10    45.623472        support    46.493283   2.4520  -3.5724
499   Topic10    45.189584       remember    46.058986   2.4518  -3.5819
470   Topic10    39.881633          prize    40.750976   2.4493  -3.7069
168   Topic10    37.103494           eyes    37.972843   2.4477  -3.7791
410   Topic10    35.404536            nra    36.274108   2.4466  -3.8260
644   Topic10    34.217325          tweet    35.086710   2.4458  -3.8601
259   Topic10    30.853108            hey    31.722493   2.4431  -3.9636
227   Topic10    30.867221          great    31.737045   2.4431  -3.9631
174   Topic10    29.672498            far    30.542052   2.4420  -4.0026
541   Topic10    27.911544       security    28.780981   2.4402  -4.0638
398   Topic10    26.728838       national    27.598663   2.4388  -4.1071
375   Topic10    26.615920         mental    27.485297   2.4387  -4.1113
675   Topic10    26.0566

In [18]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
8      11.110392        1       1  -55.649834  -69.251152
9      11.002602        1       2   66.624847   -9.902102
0      10.719315        1       3 -156.557861  150.052216
6      10.525247        1       4 -194.258713  -12.657244
3      10.112650        1       5  183.834213  -84.933601
5       9.797891        1       6   46.105312 -178.233521
2       9.776977        1       7   24.964434  189.187256
7       9.442356        1       8  157.454666   98.705948
1       9.061331        1       9 -134.184204 -182.197723
4       8.451239        1      10  -45.645641   66.279419, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
480   Default   167.000000          putin   167.000000  30.0000  30.0000
311   Default   146.000000           know   146.000000  29.0000  29.0000
524   Default   190.000000        russian   190.000000  28.0000  28.0000
681   Default   120.000000          white   120.000000  27.0000  27.0000
269   Default   119.000000          house   119.000000  26.0000  26.0000
636   Default  1340.000000          trump  1340.000000  25.0000  25.0000
98    Default   123.000000          comey   123.000000  24.0000  24.0000
387   Default   107.000000          money   107.000000  23.0000  23.0000
465   Default   152.000000      president   152.000000  22.0000  22.0000
54    Default   102.000000      billboard   102.000000  21.0000  21.0000
335   Default   194.000000           like   194.000000  20.0000  20.0000
530   Default    95.000000            say    95.000000  19.0000  19.0000
222   Default   120.000000           good   120.000000  18.0000  18.0000
567   Default   121.000000         source   121.000000  17.0000  17.0000
512   Default   112.000000          right   112.000000  16.0000  16.0000
401   Default   132.000000           need   132.000000  15.0000  15.0000
328   Default   109.000000            let   109.000000  14.0000  14.0000
357   Default    82.000000           make    82.000000  13.0000  13.0000
223   Default    78.000000            gop    78.000000  12.0000  12.0000
291   Default    72.000000  investigation    72.000000  11.0000  11.0000
224   Default    71.000000            got    71.000000  10.0000  10.0000
613   Default    77.000000          think    77.000000   9.0000   9.0000
506   Default    76.000000    republicans    76.000000   8.0000   8.0000
394   Default   174.000000        mueller   174.000000   7.0000   7.0000
441   Default   122.000000         people   122.000000   6.0000   6.0000
579   Default    74.000000          state    74.000000   5.0000   5.0000
219   Default   203.000000          going   203.000000   4.0000   4.0000
586   Default    65.000000          story    65.000000   3.0000   3.0000
631   Default    66.000000        treason    66.000000   2.0000   2.0000
527   Default    66.000000           said    66.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
591   Topic10    45.623472        support    46.493283   2.4520  -3.5724
499   Topic10    45.189584       remember    46.058986   2.4518  -3.5819
470   Topic10    39.881633          prize    40.750976   2.4493  -3.7069
168   Topic10    37.103494           eyes    37.972843   2.4477  -3.7791
410   Topic10    35.404536            nra    36.274108   2.4466  -3.8260
644   Topic10    34.217325          tweet    35.086710   2.4458  -3.8601
259   Topic10    30.853108            hey    31.722493   2.4431  -3.9636
227   Topic10    30.867221          great    31.737045   2.4431  -3.9631
174   Topic10    29.672498            far    30.542052   2.4420  -4.0026
541   Topic10    27.911544       security    28.780981   2.4402  -4.0638
398   Topic10    26.728838       national    27.598663   2.4388  -4.1071
375   Topic10    26.615920         mental    27.4